In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/690F

/content/drive/.shortcut-targets-by-id/1KB14OgLyofEEgio9tyX1hgpiVoYTq_ID/690F


In [ ]:
import string
print(type(string.punctuation))
punct=string.punctuation
print(punct)

<class 'str'>
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv("train.csv")
# test_data = pd.read_csv("test_public_expanded.csv")
# test_len = len(test_data.index)

In [ ]:
identities = ['male','female','transgender','other_gender','heterosexual','homosexual_gay_or_lesbian',
              'bisexual','other_sexual_orientation','christian','jewish','muslim','hindu','buddhist',
              'atheist','other_religion','black','white','asian','latino','other_race_or_ethnicity',
              'physical_disability','intellectual_or_learning_disability','psychiatric_or_mental_illness',
              'other_disability']

In [ ]:
# test_data = test_data[test_data['comment_text'].apply(lambda x: len(str(x)) >= 3)]
train_df = train_df[train_df['comment_text'].apply(lambda x: len(str(x)) >= 3)]

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "can not", "'cause": "because",
                       "could've": "could have", "couldn't": "could not", "didn't": "did not", 
                       "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                       "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                       "he'll": "he will", "he's": "he is", "how'd": "how did", 
                       "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                       "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                       "I'll've": "I will have","I'm": "I am", "I've": "I have",
                       "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                       "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not",
                       "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
                       "it'll've": "it will have","it's": "it is", "let's": "let us",
                       "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                       "mightn't": "might not","mightn't've": "might not have", 
                       "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                       "needn't": "need not", "needn't've": "need not have",
                       "o'clock": "of the clock", "oughtn't": "ought not", 
                       "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                       "she'd've": "she would have", "she'll": "she will",
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                       "shouldn't": "should not", "shouldn't've": "should not have", 
                       "so've": "so have","so's": "so as", "this's": "this is",
                       "that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", 
                       "there's": "there is", "here's": "here is","they'd": "they would", 
                       "they'd've": "they would have", "they'll": "they will", 
                       "they'll've": "they will have", "they're": "they are", 
                       "they've": "they have", "to've": "to have", "wasn't": "was not",
                       "we'd": "we would", "we'd've": "we would have", "we'll": 
                       "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                       "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                       "what're": "what are",  "what's": "what is", "what've": "what have", 
                       "when's": "when is", "when've": "when have", "where'd": "where did", 
                       "where's": "where is", "where've": "where have", "who'll": "who will", 
                       "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                       "why's": "why is", "why've": "why have", "will've": "will have",
                       "won't": "will not", "won't've": "will not have", "would've": "would have",
                       "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", 
                       "y'all'd": "you all would","y'all'd've": "you all would have",
                       "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                       "you'd've": "you would have", "you'll": "you will", 
                       "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

train_df['comment_text'] = train_df['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
# test_data['comment_text'] = test_data['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in punct]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
train_df['comment_text']=train_df['comment_text'].apply(lambda x: remove_punctuation(x))
# test_data['comment_text']=test_data['comment_text'].apply(lambda x: remove_punctuation(x))

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def stemi(docu):
    texts =  [t for t in docu.split() if t not in stopwords]
    doc= ' '.join(texts)
    return doc
    
stopwords = set(stopwords.words('english'))
# SnowballStemmer = SnowballStemmer("english")
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: stemi(x))
# test_data['comment_text'] = test_data['comment_text'].apply(lambda x: stemi(x))
# test_data.dropna(inplace= True)
# x = X.apply(lambda docu : stemi(docu))

# # test_data['comment_text_no_punct_split']=test_data['comment_text_no_punct'].apply(lambda x: tokenize(x.lower()))
# test_data.head()

In [ ]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: remove_stopwords(x))
# test_data['comment_text'] = test_data['comment_text'].apply(lambda x: remove_stopwords(x))

def preprocess(data):
    def clean_special_chars(text):
        punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&…'
        for p in punct:
            text = text.replace(p, ' ')
        for p in '0123456789':
            text = text.replace(p, ' ')
        #for p in "?!.,":
        #    text = text.replace(p, ' ' + p)
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x))
    return data

train_df['comment_text'] = preprocess(train_df['comment_text'])
# test_data['comment_text'] = preprocess(test_data['comment_text'])

table = str.maketrans('ᴀʙᴄᴅᴇғɢʜɪᴊᴋʟᴍɴᴏᴘʀᴛᴜᴠᴡʏᴢ', 'abcdefghijklmnoprtuvwyx')
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: x.translate(table))
# test_data['comment_text'] = test_data['comment_text'].apply(lambda x: x.translate(table))

import re
def tokenize(text):
    split=re.split("\W+",text) 
    return split
train_df['comment_text_tokenized'] = train_df['comment_text'].apply(lambda x: tokenize(x.lower()))
# test_data['comment_text_tokenized'] = test_data['comment_text'].apply(lambda x: tokenize(x.lower()))

# vectorizer = CountVectorizer()
# vectorizer.fit(train_df["comment_text"])

In [ ]:
train_df.head().loc[:, ["comment_text", "comment_text_tokenized"]]

,comment_text,comment_text_tokenized
0,This cool Its like want mother read Really gre...,"[this, cool, its, like, want, mother, read, re..."
1,Thank This life lot anxietyinducing Keep let way,"[thank, this, life, lot, anxietyinducing, keep..."
2,This urgent design problem kudos taking Very i...,"[this, urgent, design, problem, kudos, taking,..."
3,Is I able install site When releasing,"[is, i, able, install, site, when, releasing]"
4,haha guys bunch losers,"[haha, guys, bunch, losers]"


In [ ]:
# import wordcloud
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize

In [ ]:
from collections import Counter

def class_wordcloud(dataframe, label, max_words):
    # data preprocessing: concatenate all reviews per class
    all_words = []
    text = " ".join(x for x in dataframe[dataframe[label]==1].comment_text)
    wordcloud = WordCloud(max_words=max_words, collocations=False).generate(text)
    text_dictionary = wordcloud.process_text(text)
    # sort the dictionary
    word_freq={k: v for k, v in sorted(text_dictionary.items(),reverse=True, key=lambda item: item[1])}
    all_words.append(list(word_freq.keys())[:max_words])
    return all_words
    # all_words.append(word_freq)
    # plt.imshow(wordcloud, interpolation='bilinear')
    # plt.axis("off")
    # plt.title(f"Most popular {max_words} words in class {label}")
    # plt.show()

In [ ]:
# grouping identities 
groups = {
'religion' : ['atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim'],
'race' : ['asian', 'black', 'latino', 'white'],
'gender' : ['female', 'male', 'transgender'],
'sexual_identity' : ['bisexual', 'heterosexual', 'homosexual_gay_or_lesbian']
}

# train_labeled_df = train_df.loc[:, ['target'] + ['comment_text'] + identities].dropna()
toxic_df = train_df[train_df['target'] >= .5]
non_toxic_df = train_df[train_df['target'] < .5]
# labels = ['asian','black', 'christian', 'hindu',
#        'homosexual_gay_or_lesbian',
#        'jewish', 'latino', 'muslim',
#        'transgender']

ranked_lists = {}
protected_attributes = []
for group in groups.keys():
  labels = groups[group]
  all_words = []
  for label in labels:
      all_words.extend(class_wordcloud(toxic_df, label, 20))
  all_words_1 = set([str(word).lower() for word in np.array(all_words).flatten()])
  res = {}
  for word in all_words_1:
    toxic_ratio = len(toxic_df[toxic_df['comment_text'].str.contains(word)])/len(toxic_df)*100
    non_toxic_ratio = len(non_toxic_df[non_toxic_df['comment_text'].str.contains(word)])/len(non_toxic_df)*100
    res[word] = [toxic_ratio, non_toxic_ratio, toxic_ratio - non_toxic_ratio]

  res_ranked = dict(sorted(res.items(), key=lambda item: item[1][2], reverse = True))
  for i in list(res_ranked.keys())[:20]:
    protected_attributes.append(str(i))
  ranks = pd.DataFrame.from_dict(res_ranked, orient='index')
  ranked_lists[group] = ranks



In [ ]:
for group in ranked_lists.keys():
  print(group)
  ranks = ranked_lists[group]
  ranks.rename(columns={ 0: 'Toxic Ratio', 1: 'Non-Toxic Ratio', 2: 'Ratio Difference'}, inplace=True)
  print(ranks.head(10))
  print("--------------------------------------")

religion
           Toxic Ratio  Non-Toxic Ratio  Ratio Difference
white         4.975959         1.375666          3.600292
kill          4.738315         2.332693          2.405622
black         3.114997         0.852096          2.262901
people       14.426261        12.236129          2.190132
hate          3.616611         1.508908          2.107703
women         2.925853         1.669135          1.256717
gay           1.478515         0.305757          1.172758
country       4.537392         3.407602          1.129790
terrorist     1.536021         0.563808          0.972213
trump         1.726551         0.829207          0.897344
--------------------------------------
race
           Toxic Ratio  Non-Toxic Ratio  Ratio Difference
u            88.772569        84.781714          3.990855
white         4.975959         1.375666          3.600292
racist        3.927003         0.745358          3.181644
black         3.114997         0.852096          2.262901
people       14.426

In [ ]:
# finding protected attributes - identity wise
# train_labeled_df = train_df.loc[:, ['target'] + ['comment_text'] + identities].dropna()
toxic_df = train_df[train_df['target'] >= .5]
non_toxic_df = train_df[train_df['target'] < .5]
# labels = ['asian','black', 'christian', 'hindu',
#        'homosexual_gay_or_lesbian',
#        'jewish', 'latino', 'muslim',
#        'transgender']
all_words = []
labels = ['asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian',
       'jewish', 'latino', 'male', 'muslim',
       'transgender', 'white']
for label in labels:
      all_words.extend(class_wordcloud(toxic_df, label, 10))
prot_attr = set([str(word).lower() for word in np.array(all_words).flatten()])




In [ ]:
len(prot_attr)

67

In [ ]:
prot_attr = list(prot_attr)

In [ ]:
prot_attr

['transgendered',
 'sex',
 'bigotry',
 'think',
 'racist',
 'muslim',
 'live',
 'world',
 'racism',
 'priest',
 'know',
 'way',
 'terrorist',
 'black',
 'sexual',
 'rape',
 'latino',
 'man',
 'islam',
 'gay',
 'percent',
 'men',
 'straight',
 'transgender',
 'want',
 'supremacist',
 'homosexual',
 'jewish',
 'catholic',
 'heterosexual',
 'women',
 'danno',
 'church',
 'american',
 'hate',
 'hillary',
 'trump',
 'police',
 'religious',
 'country',
 'military',
 'asian',
 'partner',
 'god',
 'believe',
 'hindu',
 'buddhist',
 'chin',
 'religion',
 'intimate',
 'atheist',
 'right',
 'belief',
 'countries',
 'white',
 'gender',
 'israel',
 'people',
 'chinese',
 'male',
 'christian',
 'race',
 'bisexual',
 'jew',
 'woman',
 'illegal',
 'group']

In [ ]:
to_remove = ['american', 'belief', 'believe', 'chin', 'countries','country','danno', 'gender', 'god', 'group', 'hate', 'hillary', 'illegal', 'intimate', 'israel', 'know', 'live', 'partner', 'people', 'percent',
'police', 'rape', 'right', 'supremacist','terrorist','think','trump','want','way','world']
for word in to_remove:
  prot_attr.remove(word)
print(len(prot_attr))
prot_attr

37


['transgendered',
 'sex',
 'bigotry',
 'racist',
 'muslim',
 'racism',
 'priest',
 'black',
 'sexual',
 'latino',
 'man',
 'islam',
 'gay',
 'men',
 'straight',
 'transgender',
 'homosexual',
 'jewish',
 'catholic',
 'heterosexual',
 'women',
 'church',
 'religious',
 'military',
 'asian',
 'hindu',
 'buddhist',
 'religion',
 'atheist',
 'white',
 'chinese',
 'male',
 'christian',
 'race',
 'bisexual',
 'jew',
 'woman']

In [ ]:
train_df.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count',
       'comment_text_tokenized'],
      dtype='object')

In [ ]:
train_df["comment_text_tokenized"]
train_df["prot_word_count"] = train_df["comment_text_tokenized"].apply(lambda x: sum([x.count(word) for word in prot_attr]))


In [ ]:
def remove_id_words(s):
    s = s.lower()
    for w in protected_attributes:
        s = s.replace(w, '')
    return s

In [ ]:
train_df["edited_comment_text"] = train_df["comment_text"].apply(lambda x : remove_id_words(x))

In [ ]:
to_show = ["comment_text", "comment_text_tokenized", "edited_comment_text", "prot_word_count"]
train_df[train_df["prot_word_count"] > 3].head().loc[:, to_show]

,comment_text,comment_text_tokenized,edited_comment_text,prot_word_count
275,I conservative I ashamed Oregon taxpayer suppo...,"[i, conservative, i, ashamed, oregon, taxpayer...",i conservative i ashamed oregon taxpayer sppor...,8
852,Is thing gay trans woman Technically Theoretic...,"[is, thing, gay, trans, woman, technically, th...",is thing trans wo technically theoretically e...,6
857,Interesting article written Beth Slovic WW fai...,"[interesting, article, written, beth, slovic, ...",interesting article written beth slovic ww fai...,4
927,It little disappointing mainsplaining lobbed G...,"[it, little, disappointing, mainsplaining, lob...",it little disappointing mainsplaining lobbed g...,4
1067,The CDC giving obviously sound advice Only wom...,"[the, cdc, giving, obviously, sound, advice, o...",the cdc giving obviosly sond advice only harm...,6


In [ ]:
train_df.to_csv("final_processed_train_data.csv")